### 光流估计

光流是空间运动物体在观测成像平面上的像素运动的“瞬时速度”，根据各个像素点的速度矢量特征，可以对图像进行动态分析，例如目标跟踪。

- 亮度恒定：同一点随着时间的变化，其亮度不会发生改变。

- 小运动：随着时间的变化不会引起位置的剧烈变化，只有小运动情况下才能用前后帧之间单位位置变化引起的灰度变化去近似灰度对位置的偏导数。

- 空间一致：一个场景上邻近的点投影到图像上也是邻近点，且邻近点速度一致。因为光流法基本方程约束只有一个，而要求x，y方向的速度，有两个未知变量。所以需要连立n多个方程求解。


![title](img/lk_2.png)

![title](img/lk_1.png)

### Lucas-Kanade 算法

![title](img/lk_3.png)

如何求解方程组呢？看起来一个像素点根本不够，在物体移动过程中还有哪些特性呢？

![title](img/lk_4.png)

### cv2.calcOpticalFlowPyrLK():
参数：
- prevImage 前一帧图像

- nextImage 当前帧图像

- prevPts 待跟踪的特征点向量

- winSize 搜索窗口的大小

- maxLevel 最大的金字塔层数

返回：

- nextPts 输出跟踪特征点向量

- status 特征点是否找到，找到的状态为1，未找到的状态为0

In [12]:
import numpy as np
import cv2

cap = cv2.VideoCapture('img/test.avi')
#角点检测参数  maxCorners=100	角点数量上限 qualityLevel=0.3	角点最小质量	取值范围是 0~1，表示角点的“最差可接受质量”，minDistance=7	最小间距	指定每两个角点之间至少要有 7 个像素的距离
feature_params = dict(
    maxCorners=50,
    qualityLevel=0.3,
    minDistance=7,
)
# lucas kanade参数  winSize=(15, 15)	光流窗口大小	每次跟踪时，在特征点周围取一个 15×15 像素的小窗口，  maxLevel=2	金字塔层数	使用金字塔（Pyramid）分层方法来加速和稳定光流计算
lk_params = dict(winSize=(15, 15),
                 maxLevel=2)
color = np.random.randint(0,255,(100,3))
ret,old_frame=cap.read()
old_gray=cv2.cvtColor(old_frame,cv2.COLOR_BGR2GRAY)
# 返回所有检测特征点，需要输入图像，角点最大数量（效率），品质因子（特征值越大的越好，来筛选）
# 距离相当于这区间有比这个角点强的，就不要这个弱的了
p0=cv2.goodFeaturesToTrack(old_gray,mask=None,**feature_params)

mask=np.zeros_like(old_frame)

while True:
    _,frame=cap.read()
    if frame is None: break
    frame_gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #基于前一帧合当前帧图像,以及前一帧图像角点(特征点) err是误差，越小代表跟踪越精确
    p1,st,err=cv2.calcOpticalFlowPyrLK(old_gray,frame_gray,p0,None,**lk_params)
    #st	每个特征点的状态：1=跟踪成功，0=跟丢了
    good_new=p1[st==1]
    good_old =p0[st==1]
    #绘制轨迹
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        #ravel 拉平成一维数组，x 合y坐标
        a,b=new.ravel()
        c,d = old.ravel()
        #需要讲坐标转为int
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame=cv2.circle(frame,(int(a), int(b)),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    
    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break
    #更新旧帧为当前帧
    old_gray=frame_gray.copy()
    #只保留追踪到的
    p0=good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()